# LSA 실습

- 사이키런 제공 fetch_20 newsgroups 데이터셋 기반
  - 8개의 토픽관련 문서에 대해서 진행

In [3]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출. 
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

# 위에서 cats 변수로 기재된 category만 추출. featch_20newsgroups( )의 categories에 cats 입력
dataset= fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), 
                            categories=cats, random_state=0)

documents = dataset.data
len(documents)

7862

In [4]:
dataset.target_names

['comp.graphics',
 'comp.windows.x',
 'rec.motorcycles',
 'rec.sport.baseball',
 'sci.electronics',
 'sci.med',
 'soc.religion.christian',
 'talk.politics.mideast']

In [5]:
news_df = pd.DataFrame({'document': documents})

# 알파벳 이외의 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# 길이가 3이하인 문자 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

# 소문자로 바꾸기
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 1,000개의 단어만 추출
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # DTM의 행렬 크기 반환

(7862, 1000)

- 7862 문서에서 1,000개의 단어만 활용하여 문서-단어 행렬(DTM)을 만듬

### Truncated SVD를 활용하여 토픽 모델링

In [7]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=8, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

svd_model.components_.shape

(8, 1000)

In [8]:
svd_model.singular_values_

array([14.4015569 ,  8.5841358 ,  7.4072784 ,  6.90165227,  6.79547971,
        6.68958349,  6.38475903,  6.13175012])

In [9]:
terms = vectorizer.get_feature_names()
len(terms)

1000

In [10]:
n = 8
components = svd_model.components_
for index, topic in enumerate(components):
    print('Topic %d: '%(index + 1), [terms[i] for i in topic.argsort()[: -n - 1: -1]])

Topic 1:  ['know', 'like', 'just', 'think', 'does', 'people', 'time', 'good']
Topic 2:  ['thanks', 'program', 'graphics', 'file', 'advance', 'window', 'mail', 'help']
Topic 3:  ['israel', 'people', 'jews', 'armenian', 'israeli', 'jewish', 'armenians', 'arab']
Topic 4:  ['know', 'does', 'thanks', 'bike', 'jesus', 'advance', 'anybody', 'church']
Topic 5:  ['bike', 'ride', 'israel', 'just', 'riding', 'bikes', 'armenian', 'armenians']
Topic 6:  ['thanks', 'israel', 'year', 'game', 'armenian', 'games', 'armenians', 'team']
Topic 7:  ['israel', 'israeli', 'arab', 'jews', 'jewish', 'arabs', 'does', 'just']
Topic 8:  ['does', 'know', 'window', 'armenian', 'armenians', 'like', 'just', 'turkish']


Topic 2는 comp.windows.x 관련, Topic 3, Topic 7 'talk.politics.mideast', 'soc.religion.christian' 토픽으로 보여짐
